In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import re
import fuzzymatcher
WHITE_SPACE_PATTERN = re.compile(r' +')

# Effettuo scraping delle medaglie

In [2]:
base_url = "https://tokyo2020.coni.it/it/medagliere-italia-team.html"
print(base_url)
response = requests.get(base_url)
html_page = response.text
soup = bs(html_page)

https://tokyo2020.coni.it/it/medagliere-italia-team.html


In [3]:
mydivs = soup.find_all("div", {"class": "med-disciplina"})
disci_evento = [item.get_text() for item in mydivs]

In [4]:
mydivs = soup.find_all("div", {"class": "med-data"})
medaglie_raw = [item.get_text() for item in mydivs]
medaglie = []
for item in medaglie_raw:
    medaglie.append(item[3:].split("\n")[0].strip())

In [5]:
myspans = soup.find_all("span", {"class": "atleti"})
atleti_raw = [item.get_text() for item in myspans]
atleti = []
for item in atleti_raw:
    atleti.append(item[2:-2])

In [6]:
disci = []
evento = []
for item in disci_evento:
    disci.append(item[1:].split("\n")[0].strip())
    evento.append(item[1:].split("\n")[1].strip())

In [7]:
df = pd.DataFrame(list(zip(disci, evento, atleti, medaglie)), columns =['disciplina', 'evento', 'atleta', 'medaglia'])
df["atleta"] = df["atleta"].str.replace('\n\n\n', ', ')
df["atleta"] = df["atleta"].str.replace(WHITE_SPACE_PATTERN, ' ')
df

,disciplina,evento,atleta,medaglia
0,Ginnastica,Ritmica Conc. generale a sq.,"MARTINA CENTOFANTI, AGNESE DURANTI, ALESSIA MA...",BRONZO
1,Lotta,Libera 97 kg,ABRAHAM CONYEDO RUANO,BRONZO
2,Atletica,4 x 100 m,"ESEOSA FOSTINE DESALU, LAMONT MARCELL JACOBS, ...",ORO
3,Karate,Kumite 75 kg,LUIGI BUSA',ORO
4,Atletica,Marcia 20 km,ANTONELLA PALMISANO,ORO
5,Karate,Kata,VIVIANA BOTTARO,BRONZO
6,Ciclismo,Pista Omnium,ELIA VIVIANI,BRONZO
7,Atletica,Marcia 20 km,MASSIMO STANO,ORO
8,Canoa,K1 200m,MANFREDI RIZZA,ARGENTO
9,Nuoto di Fondo,10 Km,GREGORIO PALTRINIERI,BRONZO


# Carico il dataset con il quale fare il linkage

In [8]:
df2 = pd.read_csv('CalendarioAzzurri.csv')
df3 = df2.groupby('evento').agg(lambda x: ', '.join(x)).reset_index()[['evento','nome_intero']]
df3

,evento,nome_intero
0,10.000m U - Finale,YEMANEBERHAN CRIPPA
1,100m D - Qualificazioni,"ANNA BONGIORNI, VITTORIA FONTANA"
2,100m D - Semifinali,ANNA BONGIORNI
3,100m U - Finale,LAMONT MARCELL JACOBS
4,100m U - Qualificazioni,"LAMONT MARCELL JACOBS, FILIPPO TORTU"
...,...,...
444,Trentaduesimi e Sedicesimi Individuale U,MAURO NESPOLI
445,Triplo D - Qualificazioni,DARIYA DERKACH
446,Triplo U - Finale,"ANDREA DALLAVALLE, CHIEBUKA EMMANUEL IHEMEJE"
447,Triplo U - Qualificazione,"TOBIA BOCCHI, ANDREA DALLAVALLE, CHIEBUKA EMMA..."


In [9]:
left_on = ["evento", "atleta"]
right_on = ["evento", "nome_intero"]
matched_results = fuzzymatcher.fuzzy_left_join(df,
                                            df3,
                                            left_on,
                                            right_on)

In [10]:
matched_results

,best_match_score,__id_left,__id_right,disciplina,evento_left,atleta,medaglia,evento_right,nome_intero
0,1.417747,0_left,359_right,Ginnastica,Ritmica Conc. generale a sq.,"MARTINA CENTOFANTI, AGNESE DURANTI, ALESSIA MA...",BRONZO,Ritmica - Finale All-Around Gruppi,"MARTINA CENTOFANTI, AGNESE DURANTI, ALESSIA MA..."
54,0.793452,1_left,239_right,Lotta,Libera 97 kg,ABRAHAM CONYEDO RUANO,BRONZO,Libera - Ottavi 97 kg U,ABRAHAM CONYEDO RUANO
58,1.341956,2_left,423_right,Atletica,4 x 100 m,"ESEOSA FOSTINE DESALU, LAMONT MARCELL JACOBS, ...",ORO,Staffetta 4x100m U - Finale,"ESEOSA FOSTINE DESALU, LAMONT MARCELL JACOBS, ..."
60,0.595427,3_left,232_right,Karate,Kumite 75 kg,LUIGI BUSA',ORO,Kumite - Eliminazioni -75 kg U,LUIGI BUSÀ
62,0.266979,4_left,18_right,Atletica,Marcia 20 km,ANTONELLA PALMISANO,ORO,20km Marcia D,"ELEONORA ANNA GIORGI, ANTONELLA PALMISANO, VAL..."
63,0.324058,5_left,229_right,Karate,Kata,VIVIANA BOTTARO,BRONZO,Kata - Ranking Round D,VIVIANA BOTTARO
65,0.348310,6_left,279_right,Ciclismo,Pista Omnium,ELIA VIVIANI,BRONZO,Pista - Omnium 1/4 Scratch U,ELIA VIVIANI
69,0.270715,7_left,19_right,Atletica,Marcia 20 km,MASSIMO STANO,ORO,20km marcia U - Finale,"FRANCESCO FORTUNATO, MASSIMO STANO, FEDERICO T..."
70,0.614359,8_left,187_right,Canoa,K1 200m,MANFREDI RIZZA,ARGENTO,Finale K1 200m U,MANFREDI RIZZA
73,0.065836,9_left,9_right,Nuoto di Fondo,10 Km,GREGORIO PALTRINIERI,BRONZO,10km U,"GREGORIO PALTRINIERI, MARIO SANZULLO"


In [11]:
df['evento'] = matched_results['evento_right'].reset_index(drop=True)
df

,disciplina,evento,atleta,medaglia
0,Ginnastica,Ritmica - Finale All-Around Gruppi,"MARTINA CENTOFANTI, AGNESE DURANTI, ALESSIA MA...",BRONZO
1,Lotta,Libera - Ottavi 97 kg U,ABRAHAM CONYEDO RUANO,BRONZO
2,Atletica,Staffetta 4x100m U - Finale,"ESEOSA FOSTINE DESALU, LAMONT MARCELL JACOBS, ...",ORO
3,Karate,Kumite - Eliminazioni -75 kg U,LUIGI BUSA',ORO
4,Atletica,20km Marcia D,ANTONELLA PALMISANO,ORO
5,Karate,Kata - Ranking Round D,VIVIANA BOTTARO,BRONZO
6,Ciclismo,Pista - Omnium 1/4 Scratch U,ELIA VIVIANI,BRONZO
7,Atletica,20km marcia U - Finale,MASSIMO STANO,ORO
8,Canoa,Finale K1 200m U,MANFREDI RIZZA,ARGENTO
9,Nuoto di Fondo,10km U,GREGORIO PALTRINIERI,BRONZO


In [12]:
df['atleta'] = df['atleta'].str.split(pat=', ')

In [13]:
df = df.explode('atleta').reset_index(drop=True)

# Creo il dataset "Medaglie"

In [14]:
df

,disciplina,evento,atleta,medaglia
0,Ginnastica,Ritmica - Finale All-Around Gruppi,MARTINA CENTOFANTI,BRONZO
1,Ginnastica,Ritmica - Finale All-Around Gruppi,AGNESE DURANTI,BRONZO
2,Ginnastica,Ritmica - Finale All-Around Gruppi,ALESSIA MAURELLI,BRONZO
3,Ginnastica,Ritmica - Finale All-Around Gruppi,DANIELA MOGUREAN,BRONZO
4,Ginnastica,Ritmica - Finale All-Around Gruppi,MARTINA SANTANDREA,BRONZO
...,...,...,...,...
68,Pesi,67 kg U Gruppo B,MIRKO ZANNI,BRONZO
69,Judo,Quarti -52 kg D,ODETTE GIUFFRIDA,BRONZO
70,Ciclismo,Starda - Corsa in linea D,ELISA LONGO BORGHINI,BRONZO
71,Taekwondo,Finale -58 kg U,VITO DELL'AQUILA,ORO
